# WHOLE PIPE

In [1]:
import pandas as pd

## The file below is too large to upload on github

"BasicCompanyData-2024-04-01-part1_7.csv"

In [3]:
# import pandas as pd

# # Load a CSV file
# new_data = pd.read_csv('csv/CH_pull_april25/BasicCompanyData-2024-04-01-part1_7.csv')


/var/folders/6b/jdlfkyk94cx509cpcvq9qfx00000gn/T/ipykernel_97740/2900224363.py:4: DtypeWarning: Columns (41,42,43,44,45,46,47,48,49,50,51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  new_data = pd.read_csv('csv/CH_pull_april25/BasicCompanyData-2024-04-01-part1_7.csv')


In [4]:
# filtered_data = new_data[new_data['Accounts.AccountCategory'].isin(['TOTAL EXEMPTION FULL', 'FULL', 'UNAUDITED ABRIDGED', 'AUDITED ABRIDGED', 'MEDIUM'])]



In [2]:
# # Splitting the filtered_data DataFrame into 5 parts
# chunk_size = len(filtered_data) // 5
# chunks = [filtered_data.iloc[i:i + chunk_size] for i in range(0, len(filtered_data), chunk_size)]

# # Saving each chunk as a CSV in the 'csv' directory
# for index, chunk in enumerate(chunks):
#     chunk.to_csv(f'csv/CH_pull_april25/filtered_data_part{index+1}.csv', index=False)


/var/folders/6b/jdlfkyk94cx509cpcvq9qfx00000gn/T/ipykernel_3150/199055235.py:10: DtypeWarning: Columns (45,46,47,48,49,50,51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  third_chunk_df = pd.read_csv('csv/CH_pull_april25/filtered_data_part3.csv')


## Load one of the preprocessed chunks

In [5]:
# Creating a DataFrame from the first chunk
third_chunk_df = pd.read_csv('csv/CH_pull_april25/filtered_data_part3.csv')

/var/folders/6b/jdlfkyk94cx509cpcvq9qfx00000gn/T/ipykernel_3150/922702828.py:2: DtypeWarning: Columns (45,46,47,48,49,50,51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  third_chunk_df = pd.read_csv('csv/CH_pull_april25/filtered_data_part3.csv')


In [6]:
idxs = third_chunk_df[' CompanyNumber'].tolist()

In [4]:
len(idxs)

46178

# Run

In [ ]:
from utils_data_pipelines_uk import *

# Initialize an empty list to store failed company IDs and error types
failed_company_ids = []

# Delay needed: To stay within the 600 requests/5 minutes limit, we need a delay of 0.5 seconds per request.

# Define the directory for logging failed company IDs
log_dir = "db_pop_logs/"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)  # Create the directory if it does not exist
log_filename = f"failed_company_ids_log_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.txt"
log_filepath = os.path.join(log_dir, log_filename)  # Full path for the log file

# Iterate over each company ID in the list of indexes
for index, companyID in enumerate(idxs, start=1):
    divider()  # Print a divider for better readability in the output

    try:
        print(f'{index}. Processing: {companyID}')  # Print the current index and company ID being processed

        # First, we check the filing history to determine if the company is non-micro
        history = request_filling_history_pipe(companyID)
        add_delay(0.5)  # Add a delay to stay within the request limit

        # If the filing history does not contain any items, continue with the next iteration
        if not history.get('items'):
            print(f"No filing history items found for company ID: {companyID}, moving to next.")
            continue

        # Check if the company is a micro entity
        is_micro = identify_latest_aa_filing_micro_status(companyID, history)
        add_delay(0.5)  # Add a delay to stay within the request limit

        if not is_micro:
            # Check if the company ID exists in our database
            in_profile_table = exists_in_profile_table(companyID)
            add_delay(0.5)  # Add a delay to stay within the request limit

            if not in_profile_table:
                # Update the profile table with the company ID
                update_profile_table(companyID)
                add_delay(0.5)  # Add a delay to stay within the request limit
                
                # Update the XHTML table with the company ID and its filing history
                update_xhtml_table(companyID, history)
                add_delay(0.5)  # Add a delay to stay within the request limit
            else:
                print(f'Company is already in the db')  # Print a message if the company is already in the database
        else:
            print(f'Company is a MICRO entity, skip to next.')  # Print a message if the company is a micro entity

    except Exception as e:
        # Append the failed company ID and error message to the list
        failed_company_ids.append((companyID, str(e)))
        print(f"An error (main loop) occurred while processing company ID {companyID}: {str(e)}")
        # Write the error immediately to the log file
        with open(log_filepath, 'a') as log_file:
            log_file.write(f"Company ID: {companyID}, Error: {str(e)}\n")

# After processing all companies, print the list of failed company IDs and their errors
divider()  # Print a divider for better readability in the output
if failed_company_ids:
    print("Failed company IDs and errors:")
    for failed_id, error in failed_company_ids:
        print(f"Company ID: {failed_id}, Error: {error}")

import pickle
from datetime import datetime
import os

# Define the directory and filename for pickle
pkl_dir = "db_pop_logs/"
if not os.path.exists(pkl_dir):
    os.makedirs(pkl_dir)  # Create the directory if it does not exist
filename = f"failed_company_ids_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.pkl"

# Save the failed_company_ids list to a pickle file
with open(os.path.join(pkl_dir, filename), 'wb') as file:
    pickle.dump(failed_company_ids, file)  # Serialize the list and save it to the file


## save error logs

In [1]:
import pickle
from datetime import datetime
import os

# Define the directory and filename
pkl_dir = "db_pop_logs/"
if not os.path.exists(pkl_dir):
    os.makedirs(pkl_dir)
filename = f"failed_company_ids_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.pkl"

# Save the failed_company_ids list to a pickle file
with open(os.path.join(pkl_dir, filename), 'wb') as file:
    pickle.dump(failed_company_ids, file)

# Code to scan for the latest uploaded item

In [1]:
import boto3
from boto3.dynamodb.conditions import Key, Attr

# Initialize a session using Amazon DynamoDB
session = boto3.Session()
dynamodb = session.resource('dynamodb')

# Select your DynamoDB table
table = dynamodb.Table('company_xhtml_data')

# Initialize variables for pagination
last_evaluated_key = None
latest_item = None

# Paginate through the table to find the item with the latest timestamp
while True:
    if last_evaluated_key:
        response = table.scan(ExclusiveStartKey=last_evaluated_key)
    else:
        response = table.scan()

    # Extract items from the response
    items = response['Items']

    # Find the item with the latest timestamp in the current batch
    current_latest_item = max(items, key=lambda x: x['timestamp_date']) if items else None

    # Update the latest item if the current batch has a newer item
    if current_latest_item and (latest_item is None or current_latest_item['timestamp_date'] > latest_item['timestamp_date']):
        latest_item = current_latest_item

    # Check if there is another page of items
    last_evaluated_key = response.get('LastEvaluatedKey')
    if not last_evaluated_key:
        break

print("The latest item in the table is:")
print(latest_item)

The latest item in the table is:
{'timestamp_date': '2024-05-22 22:59:04', 'year': '2018-03-31', 'timestamp_unix': Decimal('1716415144'), 'companyID': '10040133', 's3key': 'xhtml_data/10040133/2018-03-31', 's3url': 's3://company-house-uk/xhtml_data/10040133/2018-03-31'}


## identify the list idx of latest uploaded file

In [21]:
idxs.index('10040133')

126